In [1]:
import torch
import torchvision
import numpy as np
import sys
sys.path.append("..") # 为了导入上层目录的d2lzh_pytorch
import d2lzh_pytorch as d2l

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [3]:
num_inputs = 784
num_outputs = 10

W = torch.tensor(np.random.normal(0, 0.01, (num_inputs,num_outputs)), dtype=torch.float)
b = torch.zeros(num_outputs, dtype=torch.float)
W.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [4]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(dim=1,keepdim=True)
    return X_exp / partition

In [5]:
def net(X):
    return softmax(torch.mm(X.view((-1, num_inputs)), W) +b)

In [6]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat.gather(1, y.view(-1,1)))

In [7]:
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1)==y).float().mean().item()

In [8]:
def evaluate_accuracy(data_iter,net):
    acc_sum, n=0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1)==y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [9]:
num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            if optimizer is None:
                d2l.sgd(params, lr, batch_size)
            else:                
                optimizer.step()
            
            
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1)==y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
             % (epoch+1, train_l_sum/n, train_acc_sum/n, test_acc))
        
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W, b], lr)

epoch 1, loss 0.7874, train acc 0.748, test acc 0.788
epoch 2, loss 0.5705, train acc 0.813, test acc 0.810
epoch 3, loss 0.5244, train acc 0.826, test acc 0.817
epoch 4, loss 0.5005, train acc 0.832, test acc 0.823
epoch 5, loss 0.4843, train acc 0.837, test acc 0.828


In [10]:
y_hat = torch.tensor([[0.3, 0.6, 0.1], [0.3, 0.2, 0.5]])
y = torch.LongTensor([0, 2])
y_hat.gather(1, y.view(-1, 1))
print(accuracy(y_hat, y))
print(evaluate_accuracy(test_iter, net))

0.5
0.8275
